In [19]:
import pandas as pd
import numpy as np
import pickle
#import re
#import nltk
#from datetime import datetime
#from nltk.corpus import stopwords
#from nltk.corpus import wordnet
#from nltk.stem import WordNetLemmatizer
#from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
#from nltk.tokenize import word_tokenize
#from nltk.tokenize import sent_tokenize
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import cross_val_score
#import datetime

from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import f1_score, roc_curve, auc, roc_auc_score, recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics

from keras.models import Model
import numpy as np
from keras.layers import Input, Dense, Embedding, Activation, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Concatenate, SimpleRNN,Bidirectional
from keras.callbacks import EarlyStopping

from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import warnings
warnings.filterwarnings('ignore')

## Inputs

In [2]:
train = pd.read_csv('../input/bt5153traintestbertfullsentence/train_data_bert_fullsent.csv', dtype={'label':str})
test = pd.read_csv('../input/bt5153traintestbertfullsentence/test_data_bert_fullsent.csv', dtype={'label':str})

In [3]:
with open('../input/5153-bert-feature-extraction-negative/negative_bert_dic_2D.pkl','rb') as f:
    bert_vec_negative_2d_dict= pickle.load(f)

In [4]:
with open('../input/5153-bert-feature-extraction-negative/distil_bert_dic_2D.pkl','rb') as f:
    distilbert_vec_negative_2d_dict= pickle.load(f)

In [5]:
df_bert_vec_negative_2d = pd.DataFrame({'review_id' : bert_vec_negative_2d_dict.keys(), 
                                'bert_vec_negative_2d' : bert_vec_negative_2d_dict.values()})

df_distilbert_vec_negative_2d = pd.DataFrame({'review_id' : distilbert_vec_negative_2d_dict.keys(), 
                                'distilbert_vec_negative_2d' : distilbert_vec_negative_2d_dict.values()})

In [6]:
df_distilbert_vec_negative_2d

,review_id,distilbert_vec_negative_2d
0,2,"[[[0.21047214, -0.18571095, 0.26193318, -0.090..."
1,12,"[[[0.22862941, -0.03539658, -0.038774937, -0.2..."
2,13,"[[[0.008773848, 0.073270224, 0.0376536, -0.386..."
3,14,"[[[0.2728221, 0.06639026, -0.11520613, -0.2626..."
4,23,"[[[0.25766477, -0.03353919, 0.10187317, -0.162..."
...,...,...
1518,39704,"[[[0.049246475, -0.14651816, -0.14440836, -0.4..."
1519,2002,"[[[0.06644378, -0.06282951, -0.023792485, -0.1..."
1520,38681,"[[[0.23082301, 0.0028012365, -0.036188904, -0...."
1521,18735,"[[[0.26668787, -0.07263901, -0.032762446, -0.2..."


In [7]:
train = train.merge(df_bert_vec_negative_2d, on='review_id')
train = train.merge(df_distilbert_vec_negative_2d, on='review_id')

test = test.merge(df_bert_vec_negative_2d, on='review_id')
test = test.merge(df_distilbert_vec_negative_2d, on='review_id')

In [8]:
train.head()

,review_id,review_content,review_stars,delivery,product,service,bert_avg,bert_max,bert_layer0,distil_bert_avg,distil_bert_max,distil_bert_layer0,label,bert_vec_negative_2d,distilbert_vec_negative_2d
0,9161,"Delivery took more than a week, short expiry d...",2,1,1,0,"[-0.06329992, -0.32380387, 0.35608995, -0.1268...","[0.93797976, 0.25249124, 1.1054325, 0.49609792...","[-0.19600664, -0.4481723, 0.20494524, -0.39789...","[-0.046782605, -0.27473888, 0.2780265, 0.03694...","[0.4727717, 0.23103717, 0.7197178, 0.6083572, ...","[-0.022450736, -0.34166092, 0.13424423, -0.170...",110,"[[0.008233139, -0.25179085, 0.32657608, -0.340...","[[[0.056987256, -0.2574688, 0.20414719, -0.574..."
1,8198,"Quality is so-so, loops are too huge resulting...",3,0,1,0,"[0.19020107, -0.3453789, 0.3494842, 0.14938574...","[1.1594802, 0.60506886, 1.084206, 0.8051733, 0...","[0.04546422, -0.29597518, -0.35977763, -0.4039...","[0.1399654, -0.11672436, 0.23911656, 0.1187961...","[0.8445691, 0.4961368, 0.7105234, 0.6109599, 0...","[0.12642525, -0.1857245, 0.08149249, -0.019622...",010,"[[0.2544121, 0.007479055, -0.28748313, -0.0733...","[[[0.28413913, -0.12802139, -0.0026180872, -0...."
2,5314,Received within 3days. Well packed in a carton...,4,0,1,0,"[-0.028732965, -0.14712504, 0.36105582, 0.0254...","[1.3182856, 0.7953221, 1.3915914, 0.87499446, ...","[-0.38282838, -0.22101544, 0.6115111, -0.33658...","[0.07374873, -0.01620108, 0.20269494, 0.149083...","[0.76418287, 0.48008785, 0.97139245, 0.5615782...","[0.026645288, -0.27306008, 0.20750262, 0.05986...",010,"[[0.54636973, 0.17431441, -0.31171578, -0.0738...","[[[0.3668539, 0.0733007, 0.0052855313, -0.3511..."
3,17589,Not sure if these were the authentic as I brou...,3,0,1,0,"[0.3033499, 0.02644875, 0.16410081, 0.04581299...","[1.2807441, 0.85781705, 0.80655, 0.49130562, 0...","[-0.078149214, 0.117149524, 0.04559862, -0.218...","[0.18770164, 0.114881225, 0.08257794, 0.121869...","[0.9882787, 0.5043827, 0.54232115, 0.44097498,...","[0.23179623, 0.050810635, 0.07551272, -0.15587...",010,"[[0.21767586, 0.31216693, 0.19653973, -0.22773...","[[[0.36659744, 0.14379221, 0.07114556, -0.2304..."
4,31446,its a fashion mask. no filter so no protection...,3,0,1,0,"[-0.18616518, -0.38300017, 0.52194715, 0.18413...","[0.65815246, 0.20897251, 1.0151551, 0.48899576...","[-0.29987, -0.26352054, 0.3190687, -0.22115439...","[-0.0903255, -0.1808794, 0.33290026, 0.2622123...","[0.66676205, 0.099603325, 0.8990194, 0.5398592...","[-0.34698877, -0.3470553, 0.3122651, 0.0037901...",010,"[[0.44435316, 0.07183391, -0.11268455, -0.1090...","[[[0.28633064, -0.097169034, -0.08996225, -0.2..."


In [9]:
train.iloc[0, -1][0].shape

(106, 768)

In [10]:
sequence_length = 106
vector_length = 768
input_shape = (sequence_length,vector_length)
model_input = Input(shape=input_shape)

In [11]:
# Convolutional Layer 
conv_blocks = []
#capture two-grams, 3-grams and 4 grams
filter_sizes = [2,3,4]
#for each filter, the number of filters
num_filters = 30
#loop over the different filter sizes
for sz in filter_sizes:
    # sz is the window size
    conv = Conv1D(filters=num_filters,
                  kernel_size=sz,
                  padding="valid",
                  activation="relu",
                  strides=1)(model_input)
    # Pooling Layer
    conv = GlobalMaxPooling1D()(conv)
    conv_blocks.append(conv)
# Fully-connected Layer
hiddenz = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

In [12]:
# It is binary classifcation problem. We can use sigmoid layer.
# If it is multi-class classifcaiton problem, we can use softmax layer 
model_output = Dense(3, activation="sigmoid")(hiddenz)
cnn_model = Model(model_input, model_output)
cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["AUC"])

In [13]:
print(cnn_model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 106, 768)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 105, 30)      46110       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 104, 30)      69150       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 103, 30)      92190       input_1[0][0]                    
______________________________________________________________________________________________

#### CNN Using Bert Vectors

In [14]:
train.bert_vec_negative_2d = train.bert_vec_negative_2d.apply(lambda x: np.array(x))
test.bert_vec_negative_2d = test.bert_vec_negative_2d.apply(lambda x: np.array(x))

In [15]:
train.iloc[0, -2].shape

(106, 768)

In [16]:
# Convert to numpy array
train_data = np.array(list(train['bert_vec_negative_2d']), dtype='float32')
test_data = np.array(list(test['bert_vec_negative_2d']), dtype='float32')
train_classes = np.array(train[['delivery', 'product','service']], dtype='int')
test_classes = np.array(test[['delivery', 'product','service']], dtype='int')

In [17]:
print(train_data.shape, test_data.shape)
print(train_classes.shape, test_classes.shape)

(1065, 106, 768) (458, 106, 768)
(1065, 3) (458, 3)


In [20]:
# Training
cnn_model.fit(train_data, train_classes,
          validation_split=0.1,
          batch_size=32,
          epochs=20,
          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)], 
          verbose=2)

Epoch 1/20
30/30 - 6s - loss: 0.5561 - auc: 0.8166 - val_loss: 0.4355 - val_auc: 0.8812
Epoch 2/20
30/30 - 0s - loss: 0.2579 - auc: 0.9563 - val_loss: 0.3339 - val_auc: 0.9226
Epoch 3/20
30/30 - 0s - loss: 0.1762 - auc: 0.9830 - val_loss: 0.3232 - val_auc: 0.9238
Epoch 4/20
30/30 - 0s - loss: 0.1285 - auc: 0.9931 - val_loss: 0.3120 - val_auc: 0.9286
Epoch 5/20
30/30 - 0s - loss: 0.1006 - auc: 0.9967 - val_loss: 0.3067 - val_auc: 0.9317
Epoch 6/20
30/30 - 0s - loss: 0.0766 - auc: 0.9990 - val_loss: 0.2894 - val_auc: 0.9383
Epoch 7/20
30/30 - 0s - loss: 0.0576 - auc: 0.9996 - val_loss: 0.3017 - val_auc: 0.9376
Epoch 8/20
30/30 - 0s - loss: 0.0422 - auc: 1.0000 - val_loss: 0.3048 - val_auc: 0.9366
Epoch 9/20
30/30 - 0s - loss: 0.0334 - auc: 1.0000 - val_loss: 0.3112 - val_auc: 0.9336


In [21]:
prediction_cnn_bert = cnn_model.predict(test_data)

In [22]:
test['delivery_cnn_negative_predp'] = prediction_cnn_bert[:, 0]
test['product_cnn_negative_predp'] = prediction_cnn_bert[:, 1]
test['service_cnn_negative_predp'] = prediction_cnn_bert[:, 2]

In [23]:
test['delivery_cnn_negative_pred'] = test['delivery_cnn_negative_predp'].apply(lambda x: round(x))
test['product_cnn_negative_pred'] = test['product_cnn_negative_predp'].apply(lambda x: round(x))
test['service_cnn_negative_pred'] =test['service_cnn_negative_predp'].apply(lambda x: round(x))

In [24]:
def add_pred_target_label(df):
    df['label_pred'] = df['delivery_cnn_negative_pred']*100 + df['product_cnn_negative_pred']*10 + df['service_cnn_negative_pred']
    df['label_pred'] = '00'+df['label_pred'].astype('str')
    df['label_pred'] = df['label_pred'].apply(lambda x: x[-3:])
    return df

In [25]:
#add prediction target label
test = add_pred_target_label(test)
#df_test.loc[df_test.review_id==40625, 'label'] = '010'
print(test['label_pred'].unique())

['010' '001' '110' '100' '101' '011' '000']


In [26]:
print('Result of Delivery from CNN with Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test.delivery, test.delivery_cnn_negative_predp)))
print(classification_report(test.delivery, test.delivery_cnn_negative_pred))

print('Result of Product from CNN with Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['product'], test.product_cnn_negative_predp)))
print(classification_report(test['product'], test.product_cnn_negative_pred))

print('Result of Service from CNN with Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['service'], test.service_cnn_negative_predp)))
print(classification_report(test['service'], test.service_cnn_negative_pred))

Result of Delivery from CNN with Bert vectors
AUC score: 0.936123758553665
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       351
           1       0.79      0.79      0.79       107

    accuracy                           0.90       458
   macro avg       0.86      0.86      0.86       458
weighted avg       0.90      0.90      0.90       458

Result of Product from CNN with Bert vectors
AUC score: 0.9346804511278196
              precision    recall  f1-score   support

           0       0.84      0.81      0.82       154
           1       0.90      0.92      0.91       304

    accuracy                           0.88       458
   macro avg       0.87      0.86      0.87       458
weighted avg       0.88      0.88      0.88       458

Result of Service from CNN with Bert vectors
AUC score: 0.8488624338624339
              precision    recall  f1-score   support

           0       0.86      0.93      0.89       350
           1

In [27]:
print('Combined Result from CNN with Bert vectors')
print(classification_report(test['label'], test.label_pred))

Combined Result from CNN with Bert vectors
              precision    recall  f1-score   support

         000       0.00      0.00      0.00         0
         001       0.68      0.40      0.50        67
         010       0.86      0.92      0.89       262
         011       0.29      0.18      0.22        22
         100       0.77      0.73      0.75        70
         101       0.25      0.35      0.29        17
         110       0.38      0.33      0.35        18
         111       0.00      0.00      0.00         2

    accuracy                           0.73       458
   macro avg       0.40      0.37      0.38       458
weighted avg       0.75      0.73      0.73       458



#### CNN Distil Bert Vectors

In [28]:
train.head()

,review_id,review_content,review_stars,delivery,product,service,bert_avg,bert_max,bert_layer0,distil_bert_avg,distil_bert_max,distil_bert_layer0,label,bert_vec_negative_2d,distilbert_vec_negative_2d
0,9161,"Delivery took more than a week, short expiry d...",2,1,1,0,"[-0.06329992, -0.32380387, 0.35608995, -0.1268...","[0.93797976, 0.25249124, 1.1054325, 0.49609792...","[-0.19600664, -0.4481723, 0.20494524, -0.39789...","[-0.046782605, -0.27473888, 0.2780265, 0.03694...","[0.4727717, 0.23103717, 0.7197178, 0.6083572, ...","[-0.022450736, -0.34166092, 0.13424423, -0.170...",110,"[[0.008233139, -0.25179085, 0.32657608, -0.340...","[[[0.056987256, -0.2574688, 0.20414719, -0.574..."
1,8198,"Quality is so-so, loops are too huge resulting...",3,0,1,0,"[0.19020107, -0.3453789, 0.3494842, 0.14938574...","[1.1594802, 0.60506886, 1.084206, 0.8051733, 0...","[0.04546422, -0.29597518, -0.35977763, -0.4039...","[0.1399654, -0.11672436, 0.23911656, 0.1187961...","[0.8445691, 0.4961368, 0.7105234, 0.6109599, 0...","[0.12642525, -0.1857245, 0.08149249, -0.019622...",010,"[[0.2544121, 0.007479055, -0.28748313, -0.0733...","[[[0.28413913, -0.12802139, -0.0026180872, -0...."
2,5314,Received within 3days. Well packed in a carton...,4,0,1,0,"[-0.028732965, -0.14712504, 0.36105582, 0.0254...","[1.3182856, 0.7953221, 1.3915914, 0.87499446, ...","[-0.38282838, -0.22101544, 0.6115111, -0.33658...","[0.07374873, -0.01620108, 0.20269494, 0.149083...","[0.76418287, 0.48008785, 0.97139245, 0.5615782...","[0.026645288, -0.27306008, 0.20750262, 0.05986...",010,"[[0.54636973, 0.17431441, -0.31171578, -0.0738...","[[[0.3668539, 0.0733007, 0.0052855313, -0.3511..."
3,17589,Not sure if these were the authentic as I brou...,3,0,1,0,"[0.3033499, 0.02644875, 0.16410081, 0.04581299...","[1.2807441, 0.85781705, 0.80655, 0.49130562, 0...","[-0.078149214, 0.117149524, 0.04559862, -0.218...","[0.18770164, 0.114881225, 0.08257794, 0.121869...","[0.9882787, 0.5043827, 0.54232115, 0.44097498,...","[0.23179623, 0.050810635, 0.07551272, -0.15587...",010,"[[0.21767586, 0.31216693, 0.19653973, -0.22773...","[[[0.36659744, 0.14379221, 0.07114556, -0.2304..."
4,31446,its a fashion mask. no filter so no protection...,3,0,1,0,"[-0.18616518, -0.38300017, 0.52194715, 0.18413...","[0.65815246, 0.20897251, 1.0151551, 0.48899576...","[-0.29987, -0.26352054, 0.3190687, -0.22115439...","[-0.0903255, -0.1808794, 0.33290026, 0.2622123...","[0.66676205, 0.099603325, 0.8990194, 0.5398592...","[-0.34698877, -0.3470553, 0.3122651, 0.0037901...",010,"[[0.44435316, 0.07183391, -0.11268455, -0.1090...","[[[0.28633064, -0.097169034, -0.08996225, -0.2..."


In [29]:
# reduce one dimension, (1, 86, 768) -> (86, 768)
train.loc[:, 'distilbert_vec_negative_2d'] = train.loc[:, 'distilbert_vec_negative_2d'].apply(lambda x: x[0])
test.loc[:, 'distilbert_vec_negative_2d'] = test.loc[:, 'distilbert_vec_negative_2d'].apply(lambda x: x[0])

In [30]:
test.loc[0, 'distilbert_vec_negative_2d'].shape

(106, 768)

In [31]:
# Convert to numpy array
train_data_distilbert = np.array(list(train.distilbert_vec_negative_2d), dtype='float32')
test_data_distilbert = np.array(list(test.distilbert_vec_negative_2d), dtype='float32')

# Training
cnn_model.fit(train_data_distilbert, train_classes,
          validation_split=0.1,
          batch_size=32,
          epochs=20,
          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)], 
          verbose=2)

# Prediction
prediction_cnn_distilbert = cnn_model.predict(test_data_distilbert)

Epoch 1/20
30/30 - 0s - loss: 0.1510 - auc: 0.9894 - val_loss: 0.2770 - val_auc: 0.9418
Epoch 2/20
30/30 - 0s - loss: 0.1052 - auc: 0.9970 - val_loss: 0.2778 - val_auc: 0.9411
Epoch 3/20
30/30 - 0s - loss: 0.0804 - auc: 0.9988 - val_loss: 0.2694 - val_auc: 0.9480
Epoch 4/20
30/30 - 0s - loss: 0.0614 - auc: 0.9995 - val_loss: 0.2766 - val_auc: 0.9449
Epoch 5/20
30/30 - 0s - loss: 0.0471 - auc: 0.9999 - val_loss: 0.2658 - val_auc: 0.9490
Epoch 6/20
30/30 - 0s - loss: 0.0388 - auc: 1.0000 - val_loss: 0.2870 - val_auc: 0.9418
Epoch 7/20
30/30 - 0s - loss: 0.0312 - auc: 1.0000 - val_loss: 0.2748 - val_auc: 0.9463
Epoch 8/20
30/30 - 0s - loss: 0.0279 - auc: 1.0000 - val_loss: 0.2863 - val_auc: 0.9439


In [32]:
test['delivery_cnn_distilbert_predp'] = prediction_cnn_distilbert[:, 0]
test['product_cnn_distilbert_predp'] = prediction_cnn_distilbert[:, 1]
test['service_cnn_distilbert_predp'] = prediction_cnn_distilbert[:, 2]

test['delivery_cnn_distilbert_pred'] = test['delivery_cnn_distilbert_predp'].apply(lambda x: round(x))
test['product_cnn_distilbert_pred'] = test['product_cnn_distilbert_predp'].apply(lambda x: round(x))
test['service_cnn_distilbert_pred'] =test['service_cnn_distilbert_predp'].apply(lambda x: round(x))

In [33]:
def add_pred_target_label(df):
    df['label_distilbert_pred'] = df['delivery_cnn_distilbert_pred']*100 + df['product_cnn_distilbert_pred']*10 + df['service_cnn_distilbert_pred']
    df['label_distilbert_pred'] = '00'+df['label_distilbert_pred'].astype('str')
    df['label_distilbert_pred'] = df['label_distilbert_pred'].apply(lambda x: x[-3:])
    return df

#add prediction target label
test = add_pred_target_label(test)
#df_test.loc[df_test.review_id==40625, 'label'] = '010'
print(test['label_distilbert_pred'].unique())

['010' '001' '101' '100' '011' '110' '000']


In [34]:
print('Result of Delivery from CNN with Distil Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test.delivery, test.delivery_cnn_distilbert_predp)))
print(classification_report(test.delivery, test.delivery_cnn_distilbert_pred))

print('Result of Product from CNN with Distil Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['product'], test.product_cnn_distilbert_predp)))
print(classification_report(test['product'], test.product_cnn_distilbert_pred))

print('Result of Service from CNN with Distil Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['service'], test.service_cnn_distilbert_predp)))
print(classification_report(test['service'], test.service_cnn_distilbert_pred))

Result of Delivery from CNN with Distil Bert vectors
AUC score: 0.9370823015682829
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       351
           1       0.81      0.74      0.77       107

    accuracy                           0.90       458
   macro avg       0.86      0.84      0.85       458
weighted avg       0.90      0.90      0.90       458

Result of Product from CNN with Distil Bert vectors
AUC score: 0.9373291182501708
              precision    recall  f1-score   support

           0       0.85      0.78      0.81       154
           1       0.89      0.93      0.91       304

    accuracy                           0.88       458
   macro avg       0.87      0.85      0.86       458
weighted avg       0.88      0.88      0.88       458

Result of Service from CNN with Distil Bert vectors
AUC score: 0.8520899470899472
              precision    recall  f1-score   support

           0       0.86      0.94      0.90 

In [35]:
print('Combined Result from CNN with Bert vectors')
print(classification_report(test['label'], test.label_distilbert_pred))

Combined Result from CNN with Bert vectors
              precision    recall  f1-score   support

         000       0.00      0.00      0.00         0
         001       0.66      0.43      0.52        67
         010       0.86      0.93      0.89       262
         011       0.27      0.18      0.22        22
         100       0.74      0.70      0.72        70
         101       0.33      0.29      0.31        17
         110       0.29      0.28      0.29        18
         111       0.00      0.00      0.00         2

    accuracy                           0.73       458
   macro avg       0.39      0.35      0.37       458
weighted avg       0.74      0.73      0.73       458



### RNN with Bert Vectors

In [36]:
input_shape = (sequence_length, vector_length)
model_input = Input(shape=input_shape)

In [37]:
hidden_output = Bidirectional(SimpleRNN(20))(model_input)
model_output = Dense(3, activation="sigmoid")(hidden_output)

In [38]:
rnn_model = Model(model_input, model_output)
rnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["AUC"])

In [39]:
print(rnn_model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 106, 768)]        0         
_________________________________________________________________
bidirectional (Bidirectional (None, 40)                31560     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 123       
Total params: 31,683
Trainable params: 31,683
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
# Training
rnn_model.fit(train_data, train_classes,
          validation_split=0.1,
          batch_size=32,
          epochs=20,
          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)], 
          verbose=2)

Epoch 1/20
30/30 - 9s - loss: 0.5337 - auc: 0.7846 - val_loss: 0.4983 - val_auc: 0.7991
Epoch 2/20
30/30 - 7s - loss: 0.4041 - auc: 0.8931 - val_loss: 0.5041 - val_auc: 0.8095
Epoch 3/20
30/30 - 7s - loss: 0.3507 - auc: 0.9224 - val_loss: 0.3891 - val_auc: 0.8894
Epoch 4/20
30/30 - 7s - loss: 0.3054 - auc: 0.9422 - val_loss: 0.3642 - val_auc: 0.9072
Epoch 5/20
30/30 - 6s - loss: 0.2747 - auc: 0.9540 - val_loss: 0.3503 - val_auc: 0.9131
Epoch 6/20
30/30 - 7s - loss: 0.2454 - auc: 0.9649 - val_loss: 0.3385 - val_auc: 0.9202
Epoch 7/20
30/30 - 7s - loss: 0.2289 - auc: 0.9684 - val_loss: 0.4203 - val_auc: 0.8762
Epoch 8/20
30/30 - 7s - loss: 0.2041 - auc: 0.9766 - val_loss: 0.3151 - val_auc: 0.9284
Epoch 9/20
30/30 - 7s - loss: 0.1755 - auc: 0.9832 - val_loss: 0.3375 - val_auc: 0.9206
Epoch 10/20
30/30 - 6s - loss: 0.1563 - auc: 0.9875 - val_loss: 0.3732 - val_auc: 0.9058
Epoch 11/20
30/30 - 7s - loss: 0.1540 - auc: 0.9874 - val_loss: 0.3426 - val_auc: 0.9190


In [41]:
# Prediction
prediction_rnn_bert = rnn_model.predict(test_data)

test['delivery_rnn_bert_predp'] = prediction_rnn_bert[:, 0]
test['product_rnn_bert_predp'] = prediction_rnn_bert[:, 1]
test['service_rnn_bert_predp'] = prediction_rnn_bert[:, 2]

test['delivery_rnn_bert_pred'] = test['delivery_rnn_bert_predp'].apply(lambda x: round(x))
test['product_rnn_bert_pred'] = test['product_rnn_bert_predp'].apply(lambda x: round(x))
test['service_rnn_bert_pred'] =test['service_rnn_bert_predp'].apply(lambda x: round(x))

In [42]:
def add_pred_target_label(df):
    df['label_bert_rnn_pred'] = df['delivery_rnn_bert_pred']*100 + df['product_rnn_bert_pred']*10 + df['service_rnn_bert_pred']
    df['label_bert_rnn_pred'] = '00'+df['label_bert_rnn_pred'].astype('str')
    df['label_bert_rnn_pred'] = df['label_bert_rnn_pred'].apply(lambda x: x[-3:])
    return df

#add prediction target label
test = add_pred_target_label(test)
#df_test.loc[df_test.review_id==40625, 'label'] = '010'
print(test['label_bert_rnn_pred'].unique())

['010' '101' '001' '000' '100' '011' '110']


In [43]:
print('Result of Delivery from rnn with Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test.delivery, test.delivery_rnn_bert_predp)))
print(classification_report(test.delivery, test.delivery_rnn_bert_pred))

print('Result of Product from rnn with Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['product'], test.product_rnn_bert_predp)))
print(classification_report(test['product'], test.product_rnn_bert_pred))

print('Result of Service from rnn with Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['service'], test.service_rnn_bert_predp)))
print(classification_report(test['service'], test.service_rnn_bert_pred))

Result of Delivery from rnn with Bert vectors
AUC score: 0.8922437894400511
              precision    recall  f1-score   support

           0       0.89      0.94      0.92       351
           1       0.76      0.63      0.69       107

    accuracy                           0.87       458
   macro avg       0.83      0.78      0.80       458
weighted avg       0.86      0.87      0.86       458

Result of Product from rnn with Bert vectors
AUC score: 0.9022342788790157
              precision    recall  f1-score   support

           0       0.77      0.71      0.74       154
           1       0.86      0.89      0.88       304

    accuracy                           0.83       458
   macro avg       0.82      0.80      0.81       458
weighted avg       0.83      0.83      0.83       458

Result of Service from rnn with Bert vectors
AUC score: 0.8181216931216931
              precision    recall  f1-score   support

           0       0.85      0.95      0.90       350
           

In [44]:
print('Combined Result from RNN with Bert vectors')
print(classification_report(test['label'], test.label_bert_rnn_pred))

Combined Result from RNN with Bert vectors
              precision    recall  f1-score   support

         000       0.00      0.00      0.00         0
         001       0.56      0.33      0.42        67
         010       0.81      0.92      0.86       262
         011       0.18      0.09      0.12        22
         100       0.65      0.60      0.62        70
         101       0.33      0.29      0.31        17
         110       0.25      0.11      0.15        18
         111       0.00      0.00      0.00         2

    accuracy                           0.69       458
   macro avg       0.35      0.29      0.31       458
weighted avg       0.68      0.69      0.67       458



### RNN Using Distil Bert Vectors

In [45]:
# Training
rnn_model.fit(train_data_distilbert, train_classes,
          validation_split=0.1,
          batch_size=32,
          epochs=20,
          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)], 
          verbose=2)

Epoch 1/20
30/30 - 7s - loss: 0.2801 - auc: 0.9471 - val_loss: 0.3963 - val_auc: 0.8879
Epoch 2/20
30/30 - 6s - loss: 0.2215 - auc: 0.9700 - val_loss: 0.3356 - val_auc: 0.9167
Epoch 3/20
30/30 - 7s - loss: 0.1735 - auc: 0.9827 - val_loss: 0.3473 - val_auc: 0.9158
Epoch 4/20
30/30 - 6s - loss: 0.1524 - auc: 0.9868 - val_loss: 0.3486 - val_auc: 0.9161
Epoch 5/20
30/30 - 7s - loss: 0.1297 - auc: 0.9914 - val_loss: 0.3596 - val_auc: 0.9146


In [46]:
# Prediction
prediction_rnn_distilbert = rnn_model.predict(test_data_distilbert)

test['delivery_rnn_distilbert_predp'] = prediction_rnn_distilbert[:, 0]
test['product_rnn_distilbert_predp'] = prediction_rnn_distilbert[:, 1]
test['service_rnn_distilbert_predp'] = prediction_rnn_distilbert[:, 2]

test['delivery_rnn_distilbert_pred'] = test['delivery_rnn_distilbert_predp'].apply(lambda x: round(x))
test['product_rnn_distilbert_pred'] = test['product_rnn_distilbert_predp'].apply(lambda x: round(x))
test['service_rnn_distilbert_pred'] =test['service_rnn_distilbert_predp'].apply(lambda x: round(x))

In [47]:
def add_pred_target_label(df):
    df['label_distilbert_rnn_pred'] = df['delivery_rnn_distilbert_pred']*100 + df['product_rnn_distilbert_pred']*10 + df['service_rnn_distilbert_pred']
    df['label_distilbert_rnn_pred'] = '00'+df['label_distilbert_rnn_pred'].astype('str')
    df['label_distilbert_rnn_pred'] = df['label_distilbert_rnn_pred'].apply(lambda x: x[-3:])
    return df

#add prediction target label
test = add_pred_target_label(test)
#df_test.loc[df_test.review_id==40625, 'label'] = '010'
print(test['label_distilbert_rnn_pred'].unique())

['010' '110' '101' '001' '100' '011' '000' '111']


In [48]:
print('Result of Delivery from rnn with Distil Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test.delivery, test.delivery_rnn_distilbert_predp)))
print(classification_report(test.delivery, test.delivery_rnn_distilbert_pred))

print('Result of Product from rnn with Distil Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['product'], test.product_rnn_distilbert_predp)))
print(classification_report(test['product'], test.product_rnn_distilbert_pred))

print('Result of Service from rnn with Distil Bert vectors')
print('AUC score: {}'.format(roc_auc_score(test['service'], test.service_rnn_distilbert_predp)))
print(classification_report(test['service'], test.service_rnn_distilbert_pred))

Result of Delivery from rnn with Distil Bert vectors
AUC score: 0.897196261682243
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       351
           1       0.70      0.69      0.70       107

    accuracy                           0.86       458
   macro avg       0.81      0.80      0.80       458
weighted avg       0.86      0.86      0.86       458

Result of Product from rnn with Distil Bert vectors
AUC score: 0.9117182159945317
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       154
           1       0.90      0.90      0.90       304

    accuracy                           0.86       458
   macro avg       0.85      0.85      0.85       458
weighted avg       0.86      0.86      0.86       458

Result of Service from rnn with Distil Bert vectors
AUC score: 0.8161111111111111
              precision    recall  f1-score   support

           0       0.88      0.88      0.88  

In [49]:
print('Combined Result from RNN with Distil Bert vectors')
print(classification_report(test['label'], test.label_distilbert_rnn_pred))

Combined Result from RNN with Distil Bert vectors
              precision    recall  f1-score   support

         000       0.00      0.00      0.00         0
         001       0.53      0.46      0.50        67
         010       0.84      0.86      0.85       262
         011       0.19      0.18      0.19        22
         100       0.71      0.63      0.67        70
         101       0.10      0.18      0.13        17
         110       0.23      0.17      0.19        18
         111       0.00      0.00      0.00         2

    accuracy                           0.68       458
   macro avg       0.33      0.31      0.32       458
weighted avg       0.69      0.68      0.68       458



In [50]:
test.to_csv('test_cnn_rnn_negative_pred.csv', index=False)